In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('C:\\Users\\bhavy\\PycharmProjects\\FlaskProject4\\data\\mentorfile.json').read()
intents = json.loads(data_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        w = nltk.word_tokenize(pattern)
        words.extend(w)

        documents.append((w, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)

29 documents
9 classes ['anxiety', 'boyfriend', 'career', 'depression', 'financial', 'friends', 'greeting', 'loneliness', 'money']
67 unique lemmatized words ["'m", 'achieve', 'alone', 'anxious', 'at', 'boyfriend', 'ca', 'calm', 'career', 'circle', 'do', 'doe', 'donâ€™t', 'down', 'dream', 'empty', 'feel', 'finance', 'financial', 'financially', 'find', 'friend', 'greeting', 'grow', 'handle', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'in', 'job', 'know', 'life', 'lonely', 'manage', 'managing', 'me', 'meaningless', 'money', 'motivation', 'my', "n't", 'namaste', 'need', 'no', 'one', 'out', 'overwhelmed', 'problem', 'relationship', 'stability', 'stress', 'struggle', 'struggling', 'stuck', 'support', 'there', 'to', 'understand', 'understands', 'wealth', 'what', 'with', 'work']


In [4]:
pickle.dump(words, open('texts.pkl', 'wb'))
pickle.dump(classes, open('labels.pkl', 'wb'))

training = []

output_empty = [0] * len(classes)

for doc in documents:

    bag = []

    pattern_words = doc[0]

    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)


    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [5]:

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
l
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('mentormodel.keras', hist)
print("model created")

Epoch 1/200


C:\Users\bhavy\PycharmProjects\FlaskProject4\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\bhavy\PycharmProjects\FlaskProject4\.venv\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.0751 - loss: 2.2823  
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0751 - loss: 2.3065 
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1575 - loss: 2.1909    
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1952 - loss: 2.1574 
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1891 - loss: 2.1027 
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1777 - loss: 2.1070     
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2891 - loss: 1.9914 
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3944 - loss: 1.9342 
Epoch 9/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4173 - loss: 1.8863 
Epoch 10/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4457 - loss: 1.8561 
Epoch 11/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6164 - loss: 1.6847 
Epoch 12/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4854 - loss: 

In [7]:
from tensorflow.keras.models import load_model

# Load trained model
model = load_model("mentormodel.keras")

# Load words and classes
with open("texts.pkl", "rb") as file:
    words = pickle.load(file)
with open("labels.pkl", "rb") as file:
    classes = pickle.load(file)

def clean_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, word in enumerate(words):
            if word == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    return classes[np.argmax(res)]

def get_response(intent):
    for i in intents['intents']:
        if i['tag'] == intent:
            return random.choice(i['responses'])

def chatbot_response(text):
    intent = predict_class(text)
    response = get_response(intent)
    return response

# Testing the chatbot

print(chatbot_response("i feel lonely"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
You are never truly alone. There are people who care about you, even if they have not found you yet. Keep reaching out. 


In [8]:
print(chatbot_response("i have family issues"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Dark times does not define youâ€”how you rise from them does. Start small, focus on progress, and the light will return. 
